In [9]:
test1 = """1 2 0 1 1 10.0
0.0 0.0 0
1.0 1.0 0.0"""
output1 = "0"

test2 = """3 2 2 1 3 10.0
0.0 0.0 0
5.0 5.0 10
10.0 10.0 20
1 2 100
2 3 200
1.0 1.0 0.0"""
output2 = "300"

test3 = """4 2 2 1 4 10.0
0.0 0.0 0
15.0 0.0 10
15.0 15.0 20
30.0 15.0 30
1 2 100
3 4 200
1.0 1.0 0.0"""
output3 = "-1"

test4 = """3 2 2 1 3 10.0
0.0 0.0 10
5.0 5.0 5
10.0 10.0 20
1 2 100
2 3 200
1.0 1.0 0.0"""
output4 = "-1"

test5 = """5 2 5 1 5 10.0
0.0 0.0 0
5.0 5.0 5
10.0 10.0 10
5.0 5.0 15
0.0 0.0 20
1 2 100
2 3 100
3 4 100
4 2 100
4 5 100
1.0 1.0 0.0"""
output5 = "-1"

test6 = """4 4 4 1 4 5.0
0.0 0.0 0.0 0
3.0 4.0 0.0 10
6.0 8.0 0.0 20
9.0 12.0 0.0 30
1 2 100
2 3 200
3 4 300
1 4 1000
1.0 1.0 0.0"""
output6 = "600"

test7 = """3 20 2 1 3 100.0
0.0 0.0 0.0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
1000.0 1000.0 1000.0 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
2000.0 2000.0 2000.0 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20
1 2 500
2 3 500
1.0 1.0 0.0"""
output7 = "-1"

test8 = """6 3 5 1 6 4.0
0.0 0.0 0.0 0
5.0 0.0 0.0 5
10.0 0.0 0.0 10
15.0 0.0 0.0 15
20.0 0.0 0.0 20
25.0 0.0 0.0 25
1 2 100
2 3 100
3 4 100
4 5 100
5 6 100
1.0 2.0 0.0"""
output8 = "500"

test9 = """2 2 1 1 2 10.0
0.0000001 0.0000001 0
10.0000001 10.0000001 10
1 2 100
1.0 1.41421356 0.0"""
output9 = "100"

test10 = """10 3 15 1 10 100.0
0.0 0.0 0.0 0
100.0 0.0 0.0 10
200.0 0.0 0.0 20
300.0 0.0 0.0 30
400.0 0.0 0.0 40
500.0 0.0 0.0 50
600.0 0.0 0.0 60
700.0 0.0 0.0 70
800.0 0.0 0.0 80
900.0 0.0 0.0 90
1 2 100
2 3 100
3 4 100
4 5 100
5 6 100
6 7 100
7 8 100
8 9 100
9 10 100
1 3 250
2 4 250
3 5 250
4 6 250
5 7 250
6 8 250
7 9 250
8 10 250
1.0 3.0 0.0"""
output10 = "900"

cases = [
    (test1, output1),
    (test2, output2),
    (test3, output3),
    (test4, output4),
    (test5, output5),
    (test6, output6),
    (test7, output7),
    (test8, output8),
    (test9, output9),
    (test10, output10),
]

In [12]:
import sys
import math
import heapq
from collections import defaultdict


def read_input(s: str):
    data = s.split("\n")
    idx = 0
    N, D, M, S, E, L = data[idx].split()
    N = int(N)
    D = int(D)
    M = int(M)
    S = int(S)
    E = int(E)
    L = float(L)
    idx += 1

    events = {}
    regions_set = set()
    for _ in range(N):
        parts = data[idx].strip().split()
        coords = tuple(float(x) for x in parts[:D])
        t = int(parts[D])
        region = tuple(int(math.floor(float(x) / L)) for x in parts[:D])
        events[_ + 1] = {"coords": coords, "t": t, "region": region}
        regions_set.add(region)
        idx += 1

    transitions = defaultdict(list)
    for _ in range(M):
        if idx >= len(data):
            break
        parts = data[idx].strip().split()
        if len(parts) < 3:
            idx += 1
            continue
        u, v, c = map(int, parts)
        transitions[u].append((v, c))
        idx += 1

    a, b, c_const = map(float, data[idx].strip().split())

    return N, D, M, S, E, L, events, transitions, a, b, c_const


def f_delta_t(a, b, c_const, delta_t):
    return a * (delta_t**b) + c_const


def euclidean_distance(coords1, coords2):
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(coords1, coords2)))


def solve(N, D, M, S, E, L, events, transitions, a, b, c_const):
    # Assign unique IDs to regions
    region_to_id = {}
    id_counter = 0
    for event_id in events:
        region = events[event_id]["region"]
        if region not in region_to_id:
            region_to_id[region] = id_counter
            id_counter += 1
    # Map each event to its region ID
    for event_id in events:
        events[event_id]["region_id"] = region_to_id[events[event_id]["region"]]

    # Priority queue: (total_cost, current_event_id, visited_regions_bitmask)
    heap = []
    # Initial state: cost=0, at S, visited regions includes S's region
    start_region = events[S]["region_id"]
    # To handle visited regions, use bitmask if number of regions <= 20
    # Else, use frozenset or other immutable structure
    if id_counter <= 20:
        visited = 1 << start_region
        heapq.heappush(heap, (0, S, visited))
        visited_dict = defaultdict(set)
        visited_dict[S].add(visited)
    else:
        # Use frozenset
        visited = frozenset([start_region])
        heapq.heappush(heap, (0, S, visited))
        visited_dict = defaultdict(set)
        visited_dict[S].add(visited)

    while heap:
        total_cost, current_event, visited_regions = heapq.heappop(heap)

        if current_event == E:
            return total_cost

        for neighbor, energy_cost in transitions[current_event]:
            # Check temporal constraint: t_neighbor > t_current
            if events[neighbor]["t"] <= events[current_event]["t"]:
                continue
            # Compute delta_t
            delta_t = events[neighbor]["t"] - events[current_event]["t"]
            # Compute f(delta_t)
            f_val = f_delta_t(a, b, c_const, delta_t)
            # Compute Euclidean distance
            distance = euclidean_distance(
                events[current_event]["coords"], events[neighbor]["coords"]
            )
            if distance > f_val:
                continue
            # Check if neighbor's region has been visited
            neighbor_region = events[neighbor]["region_id"]
            if id_counter <= 20:
                if visited_regions & (1 << neighbor_region):
                    continue
                new_visited = visited_regions | (1 << neighbor_region)
            else:
                if neighbor_region in visited_regions:
                    continue
                new_visited = set(visited_regions)
                new_visited.add(neighbor_region)
                new_visited = frozenset(new_visited)
            # Check if this state has been visited before
            if new_visited in visited_dict[neighbor]:
                continue
            # Add to heap
            heapq.heappush(heap, (total_cost + energy_cost, neighbor, new_visited))
            visited_dict[neighbor].add(new_visited)

    # If heap is exhausted without reaching E
    return -1


if __name__ == "__main__":
    for i, (caseX, caseY) in enumerate(cases):
        try:
            N, D, M, S, E, L, events, transitions, a, b, c_const = read_input(caseX)
            out = solve(N, D, M, S, E, L, events, transitions, a, b, c_const)
            if out != int(caseY):
                print(f"Case {i+1}: Expected {caseY}, got {out}")
        except Exception as e:
            print(f"Case {i+1}: Error: {e}")

Case 2: Expected 300, got -1
Case 6: Error: list index out of range
Case 7: Error: list index out of range
